In [1]:
import sys
import os
from tqdm import tqdm
sys.path.insert(0,'/home/olgan96/.env/lib/python3.11/site-packages/')

In [2]:
import json
import pandas as pd
import psutil
import os

In [3]:
path = '../NO_BACKUP/pharmacodb/response_data/json'

In [4]:
# i starts from 3 as previous cell (with id 1, 2) lines do not have information about experiments
# I save response data to the parquet as it is faster to process further
for i in tqdm(range(3, len(os.listdir(path)))):
    filename = 'response_cell_line_' + str(i) + '.json'
    with open(os.path.join(path, filename), 'r') as f:
        response = json.loads(f.read())
    if len(response['data']['experiments'])!=0:
        df = pd.DataFrame(response['data']['experiments'])
        frames = pd.concat([pd.concat([pd.concat([pd.concat([pd.concat([df.rename(columns={'id': 'exp_id'})[['exp_id']], pd.DataFrame.from_dict(df['compound'].values.tolist()).drop(columns=['__typename']).rename(columns={'name': 'compound'})], axis=1), pd.DataFrame.from_dict(df['tissue'].values.tolist()).drop(columns=['id', '__typename']).rename(columns={'name': 'tissue'})], axis=1), pd.DataFrame.from_dict(df['profile'].values.tolist()).drop(columns=['__typename'])], axis=1), df['dose_response']], axis=1), pd.DataFrame.from_dict(df['dataset'].values.tolist()).drop(columns=['id', '__typename']).rename(columns={'name': 'dataset'})], axis=1)
        frames['cell_line_id'] = i
#        frames.to_parquet('../NO_BACKUP/pharmacodb/response_data/' + 'response_cell_line_' + str(i) + '.parquet', index=False)
    break

  0%|          | 0/1756 [00:00<?, ?it/s]


In [5]:
response

{'data': {'experiments': [{'id': 1,
    'cell_line': {'id': 3,
     'uid': 'SW13_1529_2019',
     'name': 'SW13',
     '__typename': 'CellLineWithTissueDataset'},
    'compound': {'id': 2623,
     'uid': 'PDBC02136',
     'name': '(5Z)-7-Oxozeaenol',
     '__typename': 'Compound'},
    'tissue': {'id': 1, 'name': 'Adrenal Gland', '__typename': 'Tissue'},
    'dataset': {'id': 4, 'name': 'GDSC1', '__typename': 'Dataset'},
    'dose_response': [{'dose': 0.0390625,
      'response': 61.95134029,
      '__typename': 'DoseResponse'},
     {'dose': 0.15625, 'response': 59.28218383, '__typename': 'DoseResponse'},
     {'dose': 0.625, 'response': 35.50930656, '__typename': 'DoseResponse'},
     {'dose': 2.5, 'response': 36.51840979, '__typename': 'DoseResponse'},
     {'dose': 10, 'response': 8.48978346, '__typename': 'DoseResponse'}],
    'profile': {'HS': 0.40687059,
     'Einf': 0,
     'EC50': 0.21887573,
     'AAC': 0.59561176,
     'IC50': 0.21887573,
     'DSS1': None,
     'DSS2': None

In [19]:
# concat all saved separate dataframes
frames = pd.read_parquet('../NO_BACKUP/pharmacodb/response_data/' + 'response_cell_line_' + str(3) + '.parquet')
for i in tqdm(range(4, len(os.listdir(path)))):
    filename = 'response_cell_line_' + str(i) + '.json'
    with open(os.path.join(path, filename), 'r') as f:
        response = json.loads(f.read())
    if len(response['data']['experiments'])!=0:
        df = pd.read_parquet('../NO_BACKUP/pharmacodb/response_data/' + 'response_cell_line_' + str(i) + '.parquet')
        frames = pd.concat([frames, df])

 11%|█         | 194/1755 [00:23<03:42,  7.03it/s]/tmp/ipykernel_2077817/759909895.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  frames = pd.concat([frames, df])
 15%|█▍        | 263/1755 [00:37<04:03,  6.13it/s]/tmp/ipykernel_2077817/759909895.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  frames = pd.concat([frames, df])
 16%|█▋        | 288/1755 [00:44<05:20,  4.57it/s]/tmp/ipykernel_2077817/759909895.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA

In [20]:
frames.to_parquet('../NO_BACKUP/pharmacodb/responses.parquet', index=False)

In [21]:
#~ the memory used to concat the datasets:
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1024/1024/1024)  # in GB 

12.564319610595703


In [4]:
df = pd.read_parquet('../../NO_BACKUP/pharmacodb/responses.parquet')

In [ ]:
df